In [5]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from pickle import dump

In [6]:
df = pd.read_csv("../data/processed/arbol_diabetes.csv")

# SPLIT

In [7]:
X = df.drop(columns = ["Outcome"])
y = df["Outcome"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined